In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
import numpy as np

In [ ]:
import pandas as pd
data_path = 'clean_news.csv'
df = pd.read_csv(data_path, error_bad_lines=False)

<ipython-input-4-e32f548c7113>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(data_path, error_bad_lines=False)


In [ ]:
# Display the first 10 rows of the DataFrame
df.head(10)

,Body,Label
0,karadeniz tiyatro festivali zamanı devlet tiya...,gerçek
1,sözde torbacının haklı gururu dün gece suların...,yalan
2,buseyi aramak bin liralık cihazını alıp geldi ...,gerçek
3,west ham southampton maç özet premier ligin ha...,gerçek
4,ak partili yavuz duyurdu yskya ek dilekçe vere...,gerçek
5,onlar türkiyenin annesi sinemamızın unutulmaz ...,gerçek
6,orucu bozan durumlar neler burun damlası orucu...,gerçek
7,ankara emniyeti bayram öncesinde sahte çocuk ç...,yalan
8,şeyin i̇yisini hakettiğini düşünen genç kızın ...,yalan
9,suriye sınırına komando askeri araç sevkiyatı ...,gerçek


In [ ]:
from gensim.models import Word2Vec

# Set the length of the word vectors
vector_size = 100

# Create a Word2Vec model and train it on the text data
w2v_model = Word2Vec(df['Body'].apply(lambda x: x.split()), vector_size=vector_size, min_count=1)


In [ ]:
# Display the w2v_model object
print(w2v_model)

Word2Vec<vocab=127783, vector_size=100, alpha=0.025>


LOGISTIC REGRESSION


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Split the data into X and y.
X = df['Body']
y = df['Label']

# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data using the previously trained Word2Vec model.
X_train_vec = []
for x in X_train:
    vecs = [w2v_model.wv[word] for word in x.split() if word in w2v_model.wv.key_to_index]
    if len(vecs) == 0:
        vecs = [np.zeros(w2v_model.vector_size)]
    elif len(vecs) < 100:
        padding = [(0, 100 - len(vecs)), (0, 0)]
        vecs = np.pad(vecs, padding, mode='constant')
    X_train_vec.append(np.mean(vecs, axis=0))

X_test_vec = []
for x in X_test:
    vecs = [w2v_model.wv[word] for word in x.split() if word in w2v_model.wv.key_to_index]
    if len(vecs) == 0:
        vecs = [np.zeros(w2v_model.vector_size)]
    elif len(vecs) < 100:
        padding = [(0, 100 - len(vecs)), (0, 0)]
        vecs = np.pad(vecs, padding, mode='constant')
    X_test_vec.append(np.mean(vecs, axis=0))

X_train_vec = np.vstack(X_train_vec)
X_test_vec = np.vstack(X_test_vec)


# Create a logistic regression model
lr = LogisticRegression(max_iter=10000)

# Train the model
lr.fit(X_train_vec, y_train)

# Make predictions on the test data
y_pred_lr = lr.predict(X_test_vec)



# Calculate evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Precision:", precision_score(y_test, y_pred_lr, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_lr, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_lr, average='weighted'))

Accuracy: 0.9292929292929293
Precision: 0.9293016966459167
Recall: 0.9292929292929293
F1 score: 0.9292961408784948


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[432  32]
 [ 31 396]]


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred_lr)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

      gerçek       0.93      0.93      0.93       464
       yalan       0.93      0.93      0.93       427

    accuracy                           0.93       891
   macro avg       0.93      0.93      0.93       891
weighted avg       0.93      0.93      0.93       891



In [ ]:
# Display the contents of the X_train_vec variable
print(X_train_vec)

[[-0.18849239  0.3592665   0.25817096 ... -0.4282321   0.05479677
   0.10937496]
 [-0.14194703  0.25077128  0.21129568 ... -0.316399   -0.04705495
   0.12433407]
 [-0.28874207  0.2818498   0.1890552  ... -0.41183025  0.01404249
   0.1285924 ]
 ...
 [-0.1791871   0.42223147  0.37678063 ... -0.43035847  0.08268766
   0.17144491]
 [-0.05438788  0.10571542  0.11907933 ... -0.08246582 -0.02431335
   0.03496204]
 [-0.18374084  0.25793242  0.2885723  ... -0.40650925  0.06644879
   0.0966157 ]]


In [ ]:
# Display the contents of the X_test_vec variable
print(X_test_vec)

[[-0.11173511  0.36586532  0.07731041 ... -0.3501031   0.14639024
   0.07517733]
 [-0.16121827  0.43449017  0.20247102 ... -0.43159807  0.1251011
   0.11487765]
 [-0.18899064  0.32765332  0.16869453 ... -0.4220185   0.07097211
   0.07724896]
 ...
 [-0.12910658  0.42215168  0.19973508 ... -0.3834789   0.11193318
   0.10622127]
 [-0.18304114  0.3948866   0.27899584 ... -0.5375796  -0.06468327
   0.12130594]
 [-0.04197557  0.06069463  0.05819087 ... -0.11556819  0.00723046
   0.03402801]]


In [ ]:
# Get the number of items in the X_test_vec object
len(X_test_vec)

891

In [ ]:
# Get the number of items in the X_train_vec object
len(X_train_vec)

3564

DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a decision tree classifier
dt = DecisionTreeClassifier()

# Train the model on the vectorized training data
dt.fit(X_train_vec, y_train)

# Make predictions on the vectorized test data
y_pred_dt = dt.predict(X_test_vec)

# Calculate evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Precision:", precision_score(y_test, y_pred_dt, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_dt, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_dt, average='weighted'))

Accuracy: 0.8799102132435466
Precision: 0.8801744919905057
Recall: 0.8799102132435466
F1 score: 0.8797918199716618


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion Matrix
cm_dt = confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix:")
print(cm_dt)

# Classification Report
report_dt = classification_report(y_test, y_pred_dt)
print("Classification Report:")
print(report_dt)

Confusion Matrix:
[[418  46]
 [ 61 366]]
Classification Report:
              precision    recall  f1-score   support

      gerçek       0.87      0.90      0.89       464
       yalan       0.89      0.86      0.87       427

    accuracy                           0.88       891
   macro avg       0.88      0.88      0.88       891
weighted avg       0.88      0.88      0.88       891



RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100)

# Train the model on the vectorized training data
rf.fit(X_train_vec, y_train)

# Make predictions on the vectorized test data
y_pred_rf = rf.predict(X_test_vec)

# Calculate evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_rf, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_rf, average='weighted'))

Accuracy: 0.9281705948372615
Precision: 0.9282082007271947
Recall: 0.9281705948372615
F1 score: 0.9281471968097424


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:")
print(cm_rf)

# Classification Report
report_rf = classification_report(y_test, y_pred_rf)
print("Classification Report:")
print(report_rf)

Confusion Matrix:
[[435  29]
 [ 35 392]]
Classification Report:
              precision    recall  f1-score   support

      gerçek       0.93      0.94      0.93       464
       yalan       0.93      0.92      0.92       427

    accuracy                           0.93       891
   macro avg       0.93      0.93      0.93       891
weighted avg       0.93      0.93      0.93       891



GRADIENT BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a gradient boosting classifier
gb = GradientBoostingClassifier()

# Train the model on the vectorized training data
gb.fit(X_train_vec, y_train)

# Make predictions on the vectorized test data
y_pred_gb = gb.predict(X_test_vec)

# Calculate evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Precision:", precision_score(y_test, y_pred_gb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_gb, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_gb, average='weighted'))

Accuracy: 0.9371492704826038
Precision: 0.9373116091791784
Recall: 0.9371492704826038
F1 score: 0.9371119572482427


In [ ]:
# Confusion Matrix
cm_gb = confusion_matrix(y_test, y_pred_gb)
print("Confusion Matrix:")
print(cm_gb)

# Classification Report
report_gb = classification_report(y_test, y_pred_gb)
print("Classification Report:")
print(report_gb)

Confusion Matrix:
[[441  23]
 [ 33 394]]
Classification Report:
              precision    recall  f1-score   support

      gerçek       0.93      0.95      0.94       464
       yalan       0.94      0.92      0.93       427

    accuracy                           0.94       891
   macro avg       0.94      0.94      0.94       891
weighted avg       0.94      0.94      0.94       891



SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVC

# Create a support vector machine classifier
svm = SVC()

# Train the model on the vectorized training data
svm.fit(X_train_vec, y_train)

# Make predictions on the vectorized test data
y_pred_svm = svm.predict(X_test_vec)

# Calculate evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_svm, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_svm, average='weighted'))

Accuracy: 0.9191919191919192
Precision: 0.9194785586079853
Recall: 0.9191919191919192
F1 score: 0.919219427389362


In [ ]:
cm_svm = confusion_matrix(y_test, y_pred_svm)
report_svm = classification_report(y_test, y_pred_svm)

print("Confusion Matrix:")
print(cm_svm)
print("Classification Report:")
print(report_svm)

Confusion Matrix:
[[423  41]
 [ 31 396]]
Classification Report:
              precision    recall  f1-score   support

      gerçek       0.93      0.91      0.92       464
       yalan       0.91      0.93      0.92       427

    accuracy                           0.92       891
   macro avg       0.92      0.92      0.92       891
weighted avg       0.92      0.92      0.92       891



KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create a k-nearest neighbors classifier and train it on the vectorized training data
knn = KNeighborsClassifier()
knn.fit(X_train_vec, y_train)

# Make predictions on the vectorized test data
y_pred_knn = knn.predict(X_test_vec)

# Calculate evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Precision:", precision_score(y_test, y_pred_knn, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_knn, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_knn, average='weighted'))

Accuracy: 0.9090909090909091
Precision: 0.9108229646743139
Recall: 0.9090909090909091
F1 score: 0.9091218312905402


In [ ]:
cm_knn = confusion_matrix(y_test, y_pred_knn)
report_knn = classification_report(y_test, y_pred_knn)

print("Confusion Matrix:")
print(cm_knn)
print("Classification Report:")
print(report_knn)

Confusion Matrix:
[[410  54]
 [ 27 400]]
Classification Report:
              precision    recall  f1-score   support

      gerçek       0.94      0.88      0.91       464
       yalan       0.88      0.94      0.91       427

    accuracy                           0.91       891
   macro avg       0.91      0.91      0.91       891
weighted avg       0.91      0.91      0.91       891



NAIVE BAYES

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB

# Perform min-max scaling on the training and test data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_vec)
X_test_scaled = scaler.transform(X_test_vec)

# Apply log transformation to the scaled data
X_train_log = np.log(X_train_scaled + 1)
X_test_log = np.log(X_test_scaled + 1)

# Create and train a Naive Bayes model
nb = MultinomialNB()
nb.fit(X_train_log, y_train)

# Make predictions on the test data
y_pred_nb = nb.predict(X_test_log)

# Calculate and print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Precision:", precision_score(y_test, y_pred_nb, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_nb, average='weighted'))
print("F1 score:", f1_score(y_test, y_pred_nb, average='weighted'))

Accuracy: 0.877665544332211
Precision: 0.8813411947769709
Recall: 0.877665544332211
F1 score: 0.8776402719613048


In [ ]:
cm_nb = confusion_matrix(y_test, y_pred_nb)
report_nb = classification_report(y_test, y_pred_nb)

print("Confusion Matrix:")
print(cm_nb)
print("Classification Report:")
print(report_nb)

Confusion Matrix:
[[389  75]
 [ 34 393]]
Classification Report:
              precision    recall  f1-score   support

      gerçek       0.92      0.84      0.88       464
       yalan       0.84      0.92      0.88       427

    accuracy                           0.88       891
   macro avg       0.88      0.88      0.88       891
weighted avg       0.88      0.88      0.88       891

